<a href="https://colab.research.google.com/github/YunaW-Dev/-TensorFlow-KaggleCompetition-CatVsDog/blob/main/machine%20learning%20model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip

--2021-11-06 11:52:30--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.124.128, 172.217.212.128, 172.217.214.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.124.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘cats_and_dogs_filtered.zip’

cats_and_dogs_filte 100%[===================>]  65.43M   175MB/s    in 0.4s    

2021-11-06 11:52:30 (175 MB/s) - ‘cats_and_dogs_filtered.zip’ saved [68606236/68606236]



In [2]:
import zipfile

zip_dir= 'cats_and_dogs_filtered.zip'
zip =zipfile.ZipFile(zip_dir, 'r')
zip.extractall()
zip.close()

In [3]:
import os
base_dir= 'cats_and_dogs_filtered'
train_dir= os.path.join(base_dir, 'train')
validation_dir= os.path.join(base_dir, 'validation')
train_cats_dir= os.path.join(train_dir, 'cats')
train_dogs_dir= os.path.join(train_dir, 'dogs')
validation_cats_dir=os.path.join(validation_dir, 'cats')
validation_dogs_dir=os.path.join(validation_dir, 'dogs')

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_data=ImageDataGenerator( rescale = 1.0/255 )
test_data=ImageDataGenerator( rescale = 1.0/255 )
train_generator=train_data.flow_from_directory(train_dir,
                                                    batch_size=20,
                                                    class_mode='binary',
                                                    target_size=(150, 150))     
validation_generator=test_data.flow_from_directory(validation_dir,
                                                         batch_size=20,
                                                         class_mode  = 'binary',
                                                         target_size = (150, 150))

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [5]:
import tensorflow as tf
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2), 
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'), 
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(), 
    tf.keras.layers.Dense(1024, activation='relu'), 
    tf.keras.layers.Dense(1, activation='sigmoid')  
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 18496)             0

In [7]:
from tensorflow.keras.optimizers import RMSprop

model.compile(optimizer=RMSprop(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics = ['accuracy'])

In [10]:
# class myCallback(tf.keras.callbacks.Callback):
#       def on_epoch_end(self, epochs, logs={}):
#             if (logs.get('acc')>0.98):
#                 self.model.stop_training = True

# it is clearly overfitting
train_result = model.fit(train_generator,
                              validation_data=validation_generator,
                              steps_per_epoch=100,
                              epochs=5,
                              validation_steps=50,
                              verbose=1)

Epoch 1/5
100/100 [==============================] - 11s 112ms/step - loss: 0.5553 - accuracy: 0.7260 - val_loss: 0.8289 - val_accuracy: 0.5740
Epoch 2/5
100/100 [==============================] - 11s 110ms/step - loss: 0.4888 - accuracy: 0.7650 - val_loss: 0.5580 - val_accuracy: 0.7240
Epoch 3/5
100/100 [==============================] - 11s 109ms/step - loss: 0.3861 - accuracy: 0.8250 - val_loss: 0.6482 - val_accuracy: 0.7120
Epoch 4/5
100/100 [==============================] - 11s 110ms/step - loss: 0.2883 - accuracy: 0.8760 - val_loss: 0.6295 - val_accuracy: 0.7420
Epoch 5/5
100/100 [==============================] - 11s 109ms/step - loss: 0.1824 - accuracy: 0.9305 - val_loss: 0.8078 - val_accuracy: 0.7250


## **Because it is clealy overfitting above, some measures should be used to mitigate it. The first here is augmented data**

In [11]:
# new generator for train and test data for augmented data
train_datagen = ImageDataGenerator(
    rescale=1 / 255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.1,
    zoom_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest'
)


train_generator = train_datagen.flow_from_directory(
    directory = train_dir,
    target_size = (150, 150),
    batch_size=10,
    class_mode = 'binary'
)

validation_datagen = ImageDataGenerator(1./255)


validation_generator = validation_datagen.flow_from_directory(
        directory = validation_dir,
        target_size = (150, 150),
        batch_size = 64,
        class_mode = 'binary'
)


Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [13]:
train_result = model.fit(train_generator,
                              validation_data=validation_generator,
                              epochs=5,
                              verbose=1)

Epoch 1/5
200/200 [==============================] - ETA: 0s - loss: 0.6199 - accuracy: 0.6600

/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/image_data_generator.py:720: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


200/200 [==============================] - 23s 115ms/step - loss: 0.6199 - accuracy: 0.6600 - val_loss: 128.7380 - val_accuracy: 0.6520
Epoch 2/5
200/200 [==============================] - 23s 113ms/step - loss: 0.6094 - accuracy: 0.6825 - val_loss: 78.8363 - val_accuracy: 0.6570
Epoch 3/5
200/200 [==============================] - 23s 114ms/step - loss: 0.6035 - accuracy: 0.6925 - val_loss: 73.5780 - val_accuracy: 0.6630
Epoch 4/5
200/200 [==============================] - 23s 114ms/step - loss: 0.5860 - accuracy: 0.7000 - val_loss: 67.6062 - val_accuracy: 0.6690
Epoch 5/5
200/200 [==============================] - 23s 114ms/step - loss: 0.5890 - accuracy: 0.7035 - val_loss: 255.4559 - val_accuracy: 0.5650


## **Even though the overfiting is now not as prominent, accuracy can certainly be improved.**

## **lets use transfer learning next**